In [1]:
import spacy
from spacy.language import Language
import fr_core_news_sm
from spacy_language_detection import LanguageDetector
from sklearn.linear_model import LogisticRegression
import pandas as pd
import speech_recognition as sr
import spacy
from spacy import displacy
import srsly

from spacy.training import JsonlCorpus
import spacy
from spacy.language import Language
from spacy.matcher import Matcher, DependencyMatcher, PhraseMatcher
from spacy.tokens import Token, Span

In [2]:


# We're using a component factory because the component needs to be
# initialized with the shared vocab via the nlp object
@Language.factory("flag_loc", default_config={'path_departure':'patterns_departure.jsonl',
                                              "path_destination": 'patterns_destination.jsonl',
                                             "path_steps": 'patterns_steps.jsonl'})
def flag_destination(nlp, name, path_departure, path_steps, path_destination):
    return LocMatcher(nlp.vocab, path_departure, path_steps, path_destination)

@Language.factory("language_detector")
def get_lang_detector(nlp, name):
    return LanguageDetector()

Token.set_extension('loc', default=None, force=True)

class LocMatcher:
    def __init__(self, vocab, path_departure, path_steps, path_destination): 
        self.matcher = Matcher(vocab)
        self.patterns_departure = [[pattern for pattern in srsly.read_jsonl(path_departure)]]
        self.patterns_steps = [[pattern for pattern in srsly.read_jsonl(path_steps)]]
        self.patterns_destination = [[pattern for pattern in srsly.read_jsonl(path_destination)]]
        self.matcher.add('DEPARTURE', self.patterns_departure)
        self.matcher.add('STEPS', self.patterns_steps)
        self.matcher.add('DESTINATION', self.patterns_destination)
    def __call__(self, doc):
        # This method is invoked when the component is called on a Doc
        new_ents = []  # Collect the matched spans here
        for idx, ent in enumerate(doc.ents):
            previous_span = doc[ent.start-2: ent.start]
            match = self.matcher(previous_span)
            if match: # ensure that there are more than first term 'à' or 'de'
                new_ents.append(Span(doc, ent.start, ent.end, label=match[0][0]))
            else:
                new_ents.append(ent)
        doc.ents = new_ents
        return doc

In [5]:
nlp = spacy.load("fr_core_news_md")
nlp.add_pipe('language_detector', last=True)
nlp.add_pipe("flag_loc", last=True)  # Add component to the pipeline
phrase = 'en partant de gare de lyon je souhaite me rendre à lyon perache'
phrase1 = "j'aimerais partir de Paris et passer par biarritz pour me rendre à Marseille"
phrase2 = "Je suis à Charenton-Le-Pont et je souhaite me rendre à Vélizy"
doc = nlp(phrase2)

In [ ]:
# Utilisation de Flake8
# Programme qui capture un enregistrement via le microphone/fichier audio
# Et qui traitera les données enregistré

# Fonction de capture de son par le microphone
# Check si la capture est en francais ou pas
# Si oui retourne le contenu de la capture
# Si non vous demande de recommencer l'enregistrement


def capture_mic(microphone):
    r = sr.Recognizer()
    mic = sr.Microphone(device_index=microphone)

    with mic as source:
        r.adjust_for_ambient_noise(source)
        print("you can start to talk")
        audio = r.listen(source)
    text_data = r.recognize_google(audio, language="fr-FR")

    doc = nlp(text_data)
    detected = doc._.language
    print(detected)
    if detected['language'] == "fr":
        print("c'est bien du francais")
        print(text_data)
        return text_data
    else:
        print("ce n'est pas du francais veuillez recommencer")
        capture_mic(1)


text_to_process = capture_mic(1)

In [2]:
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

app = dash.Dash(__name__)

# assume you have a "long-form" data frame
# see https://plotly.com/python/px-arguments/ for more options
df = pd.DataFrame({
    "Fruit": ["Apples", "Oranges", "Bananas", "Apples", "Oranges", "Bananas"],
    "Amount": [4, 1, 2, 2, 4, 5],
    "City": ["SF", "SF", "SF", "Montreal", "Montreal", "Montreal"]
})

fig = px.bar(df, x="Fruit", y="Amount", color="City", barmode="group")

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),

    dcc.Graph(
        id='example-graph',
        figure=fig
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)

Dash is running on http://127.0.0.1:8050/

Dash is running on http://127.0.0.1:8050/

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: on


SystemExit: 1